In [3]:
import pathlib, requests, concurrent.futures as cf
from datetime import date, timedelta
from tqdm.auto import tqdm           # works in notebook & terminal

# ── Constants you specified ────────────────────────────────────────────────
ROOT       = pathlib.Path().resolve().parents[0]        # repo root one level up
PARCEL_DIR = ROOT / "data" / "raw" / "parcel-data"
PARCEL_DIR.mkdir(parents=True, exist_ok=True)

BASE   = "https://apps.franklincountyauditor.com/Parcel_CSV/{y}/{m:02d}/Parcel.csv"
START  = date(2018, 8, 1)                               # first NFIRS month
END    = date.today().replace(day=1)                    # latest complete month
CHUNK  = 128 * 1024                                     # 128 kB → frequent bar ticks
WORKERS = 4                                             # drop to 1 on slow links

# ── Build list of (year, month) tuples we need ────────────────────────────
todo = []
cur = START
while cur <= END:
    todo.append((cur.year, cur.month))
    # advance one month
    cur = (cur.replace(day=28) + timedelta(days=4)).replace(day=1)

# ── Single-file downloader (skip if file exists) ──────────────────────────
def download_one(ym):
    y, m = ym
    dest = PARCEL_DIR / f"parcel_{m:02d}_{y}.csv"
    if dest.exists():
        return f"skip {dest.name}"

    url = BASE.format(y=y, m=m)
    r   = requests.get(url, stream=True, timeout=60)
    if r.status_code != 200:
        return f"⚠  {y}-{m:02d} missing ({r.status_code})"

    total = int(r.headers.get("content-length", 0))
    with open(dest, "wb") as f, tqdm(
            total=total, unit="B", unit_scale=True, leave=False,
            desc=dest.name, colour="green", dynamic_ncols=True,
    ) as bar:
        for chunk in r.iter_content(chunk_size=CHUNK):
            f.write(chunk)
            bar.update(len(chunk))
    return f"✓  {dest.name}"

# ── Run downloads with a visible outer bar ────────────────────────────────
pbar = tqdm(total=len(todo), desc="Completed", unit="file")
with cf.ThreadPoolExecutor(max_workers=WORKERS) as pool:
    futures = [pool.submit(download_one, ym) for ym in todo]
    for fut in cf.as_completed(futures):
        msg = fut.result()
        tqdm.write(msg)      # prints each “skip / ✓ / ⚠” line
        pbar.update(1)
pbar.close()

print("All requested snapshots processed. Files live in:", PARCEL_DIR)

Completed:   0%|          | 0/84 [00:00<?, ?file/s]

parcel_08_2018.csv:   0%|                                                                   | 0.00/231M [00:00…

parcel_10_2018.csv:   0%|                                                                   | 0.00/232M [00:00…

parcel_11_2018.csv:   0%|                                                                   | 0.00/205M [00:00…

parcel_09_2018.csv:   0%|                                                                   | 0.00/232M [00:00…

...

✓  parcel_10_2024.csv


parcel_03_2025.csv:   0%|                                                                   | 0.00/260M [00:00…

✓  parcel_01_2025.csv


parcel_04_2025.csv:   0%|                                                                   | 0.00/260M [00:00…

✓  parcel_11_2024.csv
skip parcel_05_2025.csv
skip parcel_06_2025.csv
skip parcel_07_2025.csv
✓  parcel_02_2025.csv
✓  parcel_03_2025.csv
✓  parcel_04_2025.csv
All requested snapshots processed. Files live in: C:\Repositories\jefferson-township-run-forecasting\data\raw\parcel-data
